In [2]:
import pandas as pd
from constants import *
import string
from itertools import combinations, product, chain
import numpy as np
import time

# Helping Functions

In [3]:
def contains_duplicates(X):
    return len(np.unique(X)) != len(X)



# Importing Data

In [4]:
data = pd.read_csv('data/fav_players.csv')
data.head()
data.describe


<bound method NDFrame.describe of    firstName         lastName        pos1        pos2  include  lock  bench  \
0     Brodie           Grundy        Ruck         NaN        0     0      0   
1       Zach          Merrett  Midfielder         NaN        1     0      0   
2     Reilly          O'Brien        Ruck         NaN        1     0      0   
3        Max             Gawn        Ruck         NaN        1     1      0   
4       Rory            Laird    Defender         NaN        1     1      0   
5        Sam            Walsh  Midfielder         NaN        1     1      0   
6    Stephen         Coniglio  Midfielder         NaN        0     0      0   
7     Jayden            Short    Defender         NaN        1     1      0   
8      Caleb           Daniel    Defender         NaN        1     0      0   
9     Callum            Mills    Defender         NaN        1     0      0   
10       Dan          Houston    Defender         NaN        0     0      0   
11    Jarrod      

# Formatting data

In [5]:
# fill NaN
data['pos2'] = data['pos2'].fillna('')
data.head()

# drop players with include = 0
data = data.drop(data[data['include'] == 0].index)

# convert price to int
data['price'] = data['price'].str.strip('$k').astype('int')
data['price']

# only use players with include = 1
data = data[data['include'] == 1]
data.describe


<bound method NDFrame.describe of    firstName         lastName        pos1        pos2  include  lock  bench  \
1       Zach          Merrett  Midfielder                    1     0      0   
2     Reilly          O'Brien        Ruck                    1     0      0   
3        Max             Gawn        Ruck                    1     1      0   
4       Rory            Laird    Defender                    1     1      0   
5        Sam            Walsh  Midfielder                    1     1      0   
7     Jayden            Short    Defender                    1     1      0   
8      Caleb           Daniel    Defender                    1     0      0   
9     Callum            Mills    Defender                    1     0      0   
11    Jarrod            Witts        Ruck                    1     0      0   
12       Zac           Bailey  Midfielder     Forward        1     0      0   
13       Tom         Phillips     Forward                    1     1      0   
14       Zak      

In [38]:
# separate into postions
defenders = pd.concat([data[data['pos1'] == 'Defender'], data[data['pos2'] == 'Defender']])

forwards = pd.concat([data[data['pos1'] == 'Forward'], data[data['pos2'] == 'Forward']])

midfielders = pd.concat([data[data['pos1'] == 'Midfielder'], data[data['pos2'] == 'Midfielder']])

rucks = pd.concat([data[data['pos1'] == 'Ruck'], data[data['pos2'] == 'Ruck']])


midfielders

,firstName,lastName,pos1,pos2,include,lock,bench,price,average,last3Avg,projected,aami,category
1,Zach,Merrett,Midfielder,,1,0,0,884,115,116,111,134,4
5,Sam,Walsh,Midfielder,,1,1,0,739,96,119,93,82,3
12,Zac,Bailey,Midfielder,Forward,1,0,0,590,77,67,61,57,2
17,Josh,Dunkley,Midfielder,Forward,1,0,0,741,97,108,97,130,3
18,Tim,Taranto,Midfielder,,1,1,0,718,94,84,92,121,3
19,Rory,Sloane,Midfielder,,1,0,0,580,76,86,96,70,2
21,Jye,Caldwell,Midfielder,Forward,1,0,0,502,67,74,74,75,2
24,Matt,Rowell,Midfielder,,1,1,0,574,88,76,110,59,2
25,Tom,Green,Midfielder,,1,0,0,443,65,74,70,73,2
34,Sam,Berry,Midfielder,,1,0,1,216,47,47,47,40,1


# Find all combinations for each position


## Rucks

In [39]:
ruck_combinations = []
# only combinations with locked rucks
locked_rucks = tuple(rucks.loc[rucks['lock'] == 1].index)
print(locked_rucks)
unlocked_rucks = rucks.drop([3])
unlocked_rucks.head()

for comb in list(combinations(unlocked_rucks.index,1)):
    ruck_combinations.append(locked_rucks + comb)

print(len(ruck_combinations))



print(ruck_combinations)


(3, 49)
5
[(3, 49, 2), (3, 49, 11), (3, 49, 49), (3, 49, 53), (3, 49, 57)]


In [40]:
# check combinations for too high salary
# 16% of salary cap limit

for comb in ruck_combinations:
    salary = rucks.loc[list(comb), 'price'].sum()
    if salary > 0.10*13000:
        ruck_combinations.remove(comb)

print(len(ruck_combinations))

final_ruck_combs = []

for comb in ruck_combinations:
    final_ruck_combs.append(list(comb))


print(final_ruck_combs)


4
[[3, 49, 11], [3, 49, 49], [3, 49, 53], [3, 49, 57]]


## Midfielders

In [41]:
# separate into unlocked and locked
locked_mids = tuple(midfielders.loc[midfielders['lock'] == 1].index)
unlocked_mids = midfielders.drop(list(locked_mids))


# separate into price categories for efficiency
# unlocked
premium_mids = unlocked_mids[unlocked_mids['price'] > 650]
mid_range_mids = unlocked_mids[(unlocked_mids['price'] <=650) & (unlocked_mids['price'] > 300)]
rookie_mids = unlocked_mids[(unlocked_mids['price'] <= 300) & (unlocked_mids['bench'] == 0)]
bench_mids = unlocked_mids[(unlocked_mids['bench'] == 1)]
# locked
premium_lock = midfielders.loc[(midfielders['lock'] == 1) & (midfielders['price'] > 650)]
mid_range_lock = midfielders.loc[(midfielders['lock'] == 1) & (midfielders['price'] <=650) & (midfielders['price'] > 300)]
rookie_lock = midfielders.loc[(midfielders['lock'] == 1) & (midfielders['price'] <= 300) & (midfielders['bench'] == 0)]
bench_lock = midfielders.loc[(midfielders['lock'] == 1) & (midfielders['bench'] == 1)]


# get combinations for each category
mid_arrangement = [
    3 - len(premium_lock),
    3 - len(mid_range_lock),
    2 - len(rookie_lock),
    2 - len(bench_lock)
]

# add combinations of unlocked mids to list of locked mids

premium_mids_combs = list(combinations(premium_mids.index, mid_arrangement[0]))
premium_mids_combs = list(map(lambda x: tuple(premium_lock.index) + x, premium_mids_combs))
# only combinations with locked mid range
mid_range_mids_combs = list(combinations(mid_range_mids.index, mid_arrangement[1]))
mid_range_mids_combs = list(map(lambda x: tuple(mid_range_lock.index) + x, mid_range_mids_combs))

rookie_mids_combs = list(combinations(rookie_mids.index, mid_arrangement[2]))
rookie_mids_combs = list(map(lambda x: tuple(rookie_lock.index) + x, rookie_mids_combs))

bench_mids_combs = list(combinations(bench_mids.index, mid_arrangement[3]))
bench_mids_combs = list(map(lambda x: tuple(bench_lock.index) + x, bench_mids_combs))


midfield_combinations_array = [premium_mids_combs, mid_range_mids_combs, rookie_mids_combs, bench_mids_combs]

midfield_combinations = list(product(*midfield_combinations_array))
# 9249240
print(len(midfield_combinations))
print(midfield_combinations[:10])



# print(len(midfield_combinations))

150
[((5, 18, 1), (24, 12, 19), (35, 37), (39, 34)), ((5, 18, 1), (24, 12, 19), (35, 37), (39, 36)), ((5, 18, 1), (24, 12, 19), (35, 37), (39, 38)), ((5, 18, 1), (24, 12, 19), (35, 37), (39, 40)), ((5, 18, 1), (24, 12, 19), (35, 37), (39, 45)), ((5, 18, 1), (24, 12, 21), (35, 37), (39, 34)), ((5, 18, 1), (24, 12, 21), (35, 37), (39, 36)), ((5, 18, 1), (24, 12, 21), (35, 37), (39, 38)), ((5, 18, 1), (24, 12, 21), (35, 37), (39, 40)), ((5, 18, 1), (24, 12, 21), (35, 37), (39, 45))]


In [42]:
# remove combinations with salary > 50%
t = time.time()

# flatten combinations and convert to list
mid_combinations = []

for arrangement in midfield_combinations:
    mid_combinations.append(list(chain(*list(arrangement))))

print(mid_combinations[0:5])
final_mid_combs = []

for comb in mid_combinations:
    salary = data.loc[comb, 'price'].sum()
        
    if salary < 5000:
        final_mid_combs.append(comb)

print(time.time() - t)
len(final_mid_combs)

[[5, 18, 1, 24, 12, 19, 35, 37, 39, 34], [5, 18, 1, 24, 12, 19, 35, 37, 39, 36], [5, 18, 1, 24, 12, 19, 35, 37, 39, 38], [5, 18, 1, 24, 12, 19, 35, 37, 39, 40], [5, 18, 1, 24, 12, 19, 35, 37, 39, 45]]
0.02700638771057129


150

## Combine mid and ruck combinations

In [43]:
combined = list(product(*[final_mid_combs, final_ruck_combs]))
# print(list(chain(*mid_ruck[0])))

# combine mid and ruck arrays
mid_ruck = []
for comb in combined:
    mid_ruck.append(comb[0] + comb[1])


print(mid_ruck[0])
len(mid_ruck)

[5, 18, 1, 24, 12, 19, 35, 37, 39, 34, 3, 49, 11]


600

## Defenders

In [44]:
# separate into locked and unlocked
locked_defs = tuple(defenders.loc[defenders['lock'] == 1].index)
unlocked_defs = defenders.drop(list(locked_defs))


# separate into price categories for efficiency
# unlocked
premium_defs = unlocked_defs[unlocked_defs['price'] > 600]
mid_range_defs = unlocked_defs[(unlocked_defs['price'] <=600) & (unlocked_defs['price'] > 300)]
rookie_defs = unlocked_defs[(unlocked_defs['price'] <= 300) & (unlocked_defs['bench'] == 0)]
bench_defs = unlocked_defs[(unlocked_defs['price'] <= 300) & (unlocked_defs['bench'] == 1)]

# locked 
premium_lock = defenders.loc[(defenders['lock'] == 1) & (defenders['price'] > 600)]
mid_range_lock = defenders.loc[(defenders['lock'] == 1) & (defenders['price'] <=600) & (defenders['price'] > 300)]
rookie_lock = defenders.loc[(defenders['lock'] == 1) & (defenders['price'] <= 300) & (defenders['bench'] == 0)]
bench_lock = defenders.loc[(defenders['lock'] == 1) & (defenders['bench'] == 1)]

# get combinations for each category
# needs some adjustment for edge cases
def_arrangement = [
    3 - len(premium_lock),
    3 - len(mid_range_lock),
    0 - len(rookie_lock),
    2 - len(bench_lock)
]

# add combinations of unlocked mids to list of locked mids

premium_defs_combs = list(combinations(premium_defs.index, def_arrangement[0]))
premium_defs_combs = list(map(lambda x: tuple(premium_lock.index) + x, premium_defs_combs))

mid_range_defs_combs = list(combinations(mid_range_defs.index, def_arrangement[1]))
mid_range_defs_combs = list(map(lambda x: tuple(mid_range_lock.index) + x, mid_range_defs_combs))

rookie_defs_combs = list(combinations(rookie_defs.index, def_arrangement[2]))
rookie_defs_combs = list(map(lambda x: tuple(rookie_lock.index) + x, rookie_defs_combs))

bench_defs_combs = list(combinations(bench_defs.index, def_arrangement[3]))
bench_defs_combs = list(map(lambda x: tuple(bench_lock.index) + x, bench_defs_combs))

# create all combinations for defenders from price categories
defenders_combinations_array = [premium_defs_combs, mid_range_defs_combs, rookie_defs_combs, bench_defs_combs]

defenders_combinations = list(product(*defenders_combinations_array))
# 9249240
print(len(defenders_combinations))
print(defenders_combinations[0:5])

40
[((4, 7, 8), (26, 27, 28), (), (50, 55)), ((4, 7, 8), (26, 27, 30), (), (50, 55)), ((4, 7, 8), (26, 27, 31), (), (50, 55)), ((4, 7, 8), (26, 27, 41), (), (50, 55)), ((4, 7, 8), (26, 28, 30), (), (50, 55))]


## combine tuples into an array

In [45]:
def_combinations = []

for arrangement in defenders_combinations:
    def_combinations.append(list(chain(*list(arrangement))))


print(def_combinations[0:5])


[[4, 7, 8, 26, 27, 28, 50, 55], [4, 7, 8, 26, 27, 30, 50, 55], [4, 7, 8, 26, 27, 31, 50, 55], [4, 7, 8, 26, 27, 41, 50, 55], [4, 7, 8, 26, 28, 30, 50, 55]]


## filter out combs based on salary limit

In [46]:
final_def_combs = []

for comb in def_combinations:
    salary = data.loc[comb, 'price'].sum()
        
    if salary < 4000:
        final_def_combs.append(comb)

len(final_def_combs)

40

In [47]:
combined = list(product(*[final_def_combs, mid_ruck]))
print(len(combined))

# combine arrays
def_mid_ruck = []
for comb in combined:
    def_mid_ruck.append(comb[0] + comb[1])


print(def_mid_ruck[0:5])

24000
[[4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 34, 3, 49, 11], [4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 34, 3, 49, 49], [4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 34, 3, 49, 53], [4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 34, 3, 49, 57], [4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 36, 3, 49, 11]]


## filter out any combinations with duplicate players

In [48]:
valid_combs = []

for comb in def_mid_ruck:
    if not contains_duplicates(comb):
        valid_combs.append(comb)

print(len(valid_combs))

14400


# Forwards

In [49]:
# separate into locked and unlocked
locked = tuple(forwards.loc[(forwards['lock'] == 1)].index)
unlocked_fwds = forwards.drop(list(locked))
locked_fwds = tuple(forwards.loc[(forwards['lock'] == 1) & (forwards['bench'] == 0)].index)
onfield_unlocked_fwds = unlocked_fwds[unlocked_fwds['bench'] == 0]
print(len(unlocked_fwds))
print(len(locked_fwds))

# separate into price categories for efficiency
# unlocked
premium_fwds = unlocked_fwds[unlocked_fwds['price'] > 600]
mid_range_fwds = unlocked_fwds[(unlocked_fwds['price'] <=600) & (unlocked_fwds['price'] > 300)]
rookie_fwds = unlocked_fwds[(unlocked_fwds['price'] <= 300) & (unlocked_fwds['bench'] == 0)]
bench_fwds = unlocked_fwds[(unlocked_fwds['price'] <= 300) & (unlocked_fwds['bench'] == 1)]

# locked 
premium_lock = forwards.loc[(forwards['lock'] == 1) & (forwards['price'] > 600)]
mid_range_lock = forwards.loc[(forwards['lock'] == 1) & (forwards['price'] <=600) & (forwards['price'] > 300)]
rookie_lock = forwards.loc[(forwards['lock'] == 1) & (forwards['price'] <= 300) & (forwards['bench'] == 0)]
bench_lock = forwards.loc[(forwards['lock'] == 1) & (forwards['bench'] == 1)]

# get combinations for each category
# needs some adjustment for edge cases
fwd_arrangement = [
    2 - len(premium_lock),
    3 - len(mid_range_lock),
    1 - len(rookie_lock),
    2 - len(bench_lock)
]

# add combinations of unlocked mids to list of locked mids

# premium_fwds_combs = list(combinations(premium_fwds.index, fwd_arrangement[0]))
# premium_fwds_combs = list(map(lambda x: tuple(premium_lock.index) + x, premium_fwds_combs))

# mid_range_fwds_combs = list(combinations(mid_range_fwds.index, fwd_arrangement[1]))
# mid_range_fwds_combs = list(map(lambda x: tuple(mid_range_lock.index) + x, mid_range_fwds_combs))

# rookie_fwds_combs = list(combinations(rookie_fwds.index, fwd_arrangement[2]))
# rookie_fwds_combs = list(map(lambda x: tuple(rookie_lock.index) + x, rookie_fwds_combs))
# testing
onfield_fwds = list(combinations(onfield_unlocked_fwds.index, 6 - len(locked_fwds)))
onfield_fwds = list(map(lambda x: locked_fwds + x, onfield_fwds))

bench_fwds_combs = list(combinations(bench_fwds.index, fwd_arrangement[3]))
bench_fwds_combs = list(map(lambda x: tuple(bench_lock.index) + x, bench_fwds_combs))

# create all combinations for forwards from price categories
# forwards_combinations_array = [premium_fwds_combs, mid_range_fwds_combs, rookie_fwds_combs, bench_fwds_combs]
forwards_combinations_array = [onfield_fwds, bench_fwds_combs]

forwards_combinations = list(product(*forwards_combinations_array))
# 9249240
print(len(forwards_combinations))
print(forwards_combinations[0:5])

13
3
330
[((13, 14, 47, 20, 23, 29), (48, 46)), ((13, 14, 47, 20, 23, 29), (48, 45)), ((13, 14, 47, 20, 23, 32), (48, 46)), ((13, 14, 47, 20, 23, 32), (48, 45)), ((13, 14, 47, 20, 23, 12), (48, 46))]


In [50]:
print(locked_fwds)

(13, 14, 47)


## format and filtering

In [51]:
fwd_combinations = []

for arrangement in forwards_combinations:
    fwd_combinations.append(list(chain(*list(arrangement))))


print(fwd_combinations[0:5])

[[13, 14, 47, 20, 23, 29, 48, 46], [13, 14, 47, 20, 23, 29, 48, 45], [13, 14, 47, 20, 23, 32, 48, 46], [13, 14, 47, 20, 23, 32, 48, 45], [13, 14, 47, 20, 23, 12, 48, 46]]


In [52]:
final_fwd_combs = []

for comb in fwd_combinations:
    salary = data.loc[comb, 'price'].sum()
        
    if salary < 3300:
        final_fwd_combs.append(comb)

len(final_fwd_combs)

205

# full team combinations

In [53]:
full_team = list(product(*[final_fwd_combs, valid_combs]))
print(len(full_team))


# combine arrays
full_team_combined = []
for comb in full_team:
    full_team_combined.append(comb[0] + comb[1])


print(full_team_combined[0:5])


2952000
[[13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 34, 3, 49, 11], [13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 34, 3, 49, 53], [13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 34, 3, 49, 57], [13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 36, 3, 49, 11], [13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, 27, 28, 50, 55, 5, 18, 1, 24, 12, 19, 35, 37, 39, 36, 3, 49, 53]]


In [54]:
# remove duplicates
final_teams = []

for comb in full_team_combined:
    if not contains_duplicates(comb):
        final_teams.append(comb)

print(len(final_teams))


1521900


## removing teams with too high salary

In [55]:
# teams_df = pd.DataFrame()
team_salaries = []
teams = []
projected = []
average = []
last3Avg = []
aami = []

print(data.loc[final_teams[0], ['average', 'projected', 'last3Avg']].sum().average)

for team in final_teams:
    salary = data.loc[team, 'price'].sum()

    if salary <= 12830 and salary >= 12790:
        # valid salary 29 players
        teams.append(team)
        team_salaries.append(salary)
        # add sum of scores too
        sums = data.loc[team, ['average', 'projected', 'last3Avg', 'aami']].sum()
        average.append(sums.average)
        projected.append(sums.projected)
        last3Avg.append(sums.last3Avg)
        aami.append(sums.aami)

print(len(teams))


1929
72103


In [56]:
# data frame with results
results_data = {
    'salary' : team_salaries,
    'average' : average,
    'projected' : projected,
    'last3Avg' : last3Avg,
    'aami' : aami,
    'team' : teams
}

results = pd.DataFrame(results_data, columns=['salary', 'average', 'projected', 'last3Avg', 'aami', 'team'])

results.head()

,salary,average,projected,last3Avg,aami,team
0,12799,1887,1888,1947,2424,"[13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, ..."
1,12829,1889,1890,1949,2414,"[13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, ..."
2,12809,1885,1886,1945,2453,"[13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, ..."
3,12824,1894,1891,1945,2486,"[13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, ..."
4,12808,1893,1890,1944,2476,"[13, 14, 47, 20, 23, 29, 48, 46, 4, 7, 8, 26, ..."


# Sorting teams based on project score

In [57]:
sort_by_projected = results.sort_values('projected', ascending=False)
sort_by_average = results.sort_values('average', ascending=False)
sort_by_last3Avg = results.sort_values('last3Avg', ascending=False)
sort_by_aami = results.sort_values('aami', ascending=False)

sort_by_aami.head()

,salary,average,projected,last3Avg,aami,team
3829,12803,1906,1913,1986,2584,"[13, 14, 47, 20, 23, 22, 48, 45, 4, 7, 8, 26, ..."
3839,12797,1899,1925,1990,2577,"[13, 14, 47, 20, 23, 22, 48, 45, 4, 7, 8, 26, ..."
3830,12803,1906,1913,1986,2574,"[13, 14, 47, 20, 23, 22, 48, 45, 4, 7, 8, 26, ..."
21653,12817,1909,1903,1961,2574,"[13, 14, 47, 20, 22, 42, 48, 45, 4, 7, 8, 26, ..."
9875,12822,1912,1922,1954,2571,"[13, 14, 47, 20, 29, 22, 48, 45, 4, 7, 8, 26, ..."


# display best teams for each value

In [58]:
best_projected = data.loc[sort_by_projected.iloc[0].team]
best_average = data.loc[sort_by_average.iloc[0].team]
best_last3Avg = data.loc[sort_by_last3Avg.iloc[0].team]
best_aami = data.loc[sort_by_aami.iloc[0].team]
print(sort_by_aami.iloc[0].salary)
print(sort_by_aami.iloc[0].aami)
 


12803
2584


In [59]:
# write to xlsx
with pd.ExcelWriter('../csv/best_teams.xlsx') as writer:
    best_aami.to_excel(writer, sheet_name = 'aami')
    best_projected.to_excel(writer, sheet_name = 'projected')
    best_average.to_excel(writer, sheet_name = 'average')
    best_last3Avg.to_excel(writer, sheet_name = 'last3Avg')